# 1. Import Modules

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from graph.wiki import get_encodings
from rankingmodel.data_preprocess import DataPreprocess
from rankingmodel.models import TemporalSAGE
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pickle

# 2. Load Data

In [4]:
with open('data/nasdaq_test_price.pickle', 'rb') as f:
    nasdaq_data = pickle.load(f)

In [5]:
nasdaq_data.head()

,AABA,AAON,AAPL,AAWW,AAXJ,ABAX,ABCB,ABMD,ACGL,ACHC,...,JAZZ,JBHT,JBLU,JBSS,JCOM,JJSF,JKHY,JKI,JMBA,JOBS
date,,,,,,,,,,,,,,,,,,,,,
2013-01-02,5.516981,8.894916,17.011521,45.58,53.233575,36.289190,11.861925,13.6900,14.793333,24.3700,...,54.58,56.432451,5.90,13.594767,27.844603,57.646466,36.760707,23.157976,11.750,24.050
2013-01-03,5.434556,9.049646,16.796670,44.76,52.828462,36.193238,11.824944,13.4500,14.750000,24.1900,...,55.21,56.635844,5.95,13.601933,27.757014,57.969114,36.814701,23.229510,11.750,24.315
2013-01-04,5.456536,9.095647,16.328928,45.19,52.828462,36.135666,11.926644,13.4100,14.876667,23.6300,...,55.62,57.209044,5.95,13.530268,27.923434,58.049776,36.895691,23.434664,12.400,26.510
2013-01-07,5.330151,8.995281,16.232875,44.84,52.414729,35.962952,11.899924,13.2495,14.730000,24.1500,...,55.50,56.783767,5.97,13.644932,27.809568,56.822548,36.958684,23.316236,12.499,26.400
2013-01-08,5.401586,9.016191,16.276564,45.53,51.975139,36.452309,11.926644,13.2100,14.750000,24.9392,...,56.01,56.700561,5.94,13.609099,27.888398,56.848808,36.787704,23.197095,12.700,26.255


# 3. Preprocess Data

In [6]:
DP = DataPreprocess()
window_size = 30
X_train, X_val, X_test, y_train, y_val, y_test = DP.get_data(nasdaq_data, window_size, split_ratio=(0.6,0.2,0.2))

In [7]:
print("X_train shape: ", X_train.shape)
print("y_train shape: ", y_train.shape)

X_train shape:  (481, 30, 736)
y_train shape:  (481, 736)


In [8]:
model_tickers = nasdaq_data.columns.tolist()
encoding, binary_encoding = get_encodings('20180105', 'NASDAQ', model_tickers)

In [9]:
print("Encoding shape: ", encoding.shape)
print("Binary encoding shape:, ", binary_encoding.shape)

Encoding shape:  (481, 481, 43)
Binary encoding shape:,  (481, 481)


In [10]:
batch_size = 32

X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train)

train_data = []
for i in tqdm(range(X_train_tensor.shape[2])):
    train_data.append([X_train_tensor[:,:,i], y_train_tensor[:,i]])
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, pin_memory=True)

X_valid_tensor = torch.FloatTensor(X_val)
y_valid_tensor = torch.FloatTensor(y_val)

valid_data = []
for i in tqdm(range(X_valid_tensor.shape[2])):
    valid_data.append([X_valid_tensor[:,:,i], y_valid_tensor[:,i]])
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size, shuffle=False, pin_memory=True)

100%|██████████| 246/246 [00:00<00:00, 93740.24it/s]


# 4. Train Model

In [11]:
cuda = torch.cuda.is_available() 
if cuda:
    device = 'cuda:0'
else:
    device = 'cpu'
cuda, device

(True, 'cuda:0')

In [12]:
# Testing Purpose
seq_embed_size = 64
input_size = X_train.shape[2]


input_data = torch.FloatTensor(X_train).to(device)
rel_encoding = torch.FloatTensor(encoding).to(device)


model = TemporalSAGE(
    input_size=input_size, 
    seq_embed_size=seq_embed_size, 
    relational_encoding=rel_encoding, 
    k_hops=3, 
    hop_layers=2, 
    device=device
)

In [13]:
print(model)

TemporalSAGE(
  (sequential_embedding_model): SequentialEmbedding(
    (lstm): LSTM(736, 64, num_layers=2)
  )
  (relational_embedding_model): RelationalEmbedding(
    (softmax_dim_1): Softmax(dim=1)
    (activation_function): LeakyReLU(negative_slope=0.01)
    (linear_1_hop_layer_1): Linear(in_features=171, out_features=171, bias=True)
    (linear_1_hop_layer_2): Linear(in_features=171, out_features=1, bias=True)
    (linear_2_hop_layer_1): Linear(in_features=171, out_features=171, bias=True)
    (linear_2_hop_layer_2): Linear(in_features=171, out_features=1, bias=True)
    (linear_3_hop_layer_1): Linear(in_features=171, out_features=171, bias=True)
    (linear_3_hop_layer_2): Linear(in_features=171, out_features=1, bias=True)
  )
  (combined_prediction_model): FullyConnected(
    (activation_function): LeakyReLU(negative_slope=0.01)
    (linear_layer_1): Linear(in_features=128, out_features=128, bias=True)
    (linear_layer_2): Linear(in_features=128, out_features=128, bias=True)
   

In [14]:
path = f'models/LSTM_{seq_embed_size}_seq_embed_size_{window_size}_window'
print(path)

num_epoch = int(1e4)
lr = 1e-4
weight_decay = 1e-8
criterion = nn.CrossEntropyLoss()

models/LSTM_64_seq_embed_size_30_window


In [81]:
%%time

optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

loss_array_train = []
loss_array_valid = []
patience = 0
min_loss = np.inf

for e in range(num_epoch):
    loss_array_train_tmp = []
    model.train()
    
#     for X_train_batch, Y_train_batch in train_loader:
        
    out = model(X_train_tensor.to(device))

    loss = criterion(out, y_train_tensor.to(device))
    loss_array_train_tmp.append(loss.item())

    model.zero_grad()
    loss.backward()

    torch.nn.utils.clip_grad_value_(model.parameters(), 1.)

    optimizer.step()

    loss_array_train.append(np.mean(loss_array_train_tmp))
        
    torch.cuda.empty_cache() ## 캐시 비워주기 자주 해줘야함
    
    loss_array_valid_tmp = []
    
    model.eval()
    with torch.no_grad():
#         for X_train_batch, Y_train_batch in valid_loader:
            
        out = model(X_valid_tensor.to(device))

        loss = criterion(out, y_valid_tensor.to(device))
        loss_array_valid_tmp.append(loss.item())

        loss_array_valid.append(np.mean(loss_array_valid_tmp))

    if e % 100 == 0: 
        print('Epoch: {}, Train Loss: {:.4e}, Valid Loss: {:.4e}'.format(e, loss_array_train[-1], loss_array_valid[-1]))

    ## update the minimum loss
    if min_loss > loss_array_train[-1]:
        patience = 0
        min_loss = loss_array_train[-1]
        torch.save(model.state_dict(), path)
    else:
        patience += 1

    ## early stop when patience become larger than 10
    if patience > 10:
        break
        
    torch.cuda.empty_cache() ## 캐시 비워주기 자주 해줘야함
    

plt.plot(loss_array_train, label='Train Loss')
plt.plot(loss_array_valid, label='Valid Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.yscale('log')
plt.legend()
plt.show()

torch.cuda.empty_cache()

RuntimeError: Expected object of scalar type Long but got scalar type Float for argument #2 'target' in call to _thnn_nll_loss_forward